# 1 Prepear Work

In [1]:
import os.path, jieba

In [2]:
import pandas as pd

In [3]:
word_content_path = r'D:\Assignment\Project_01\news-word-cut.txt'

In [4]:
sentence_content_path = r'D:\Assignment\Project_01\news-sentence-cut.txt'

In [5]:
sentence_content_with_punctuation_path = r'D:\Assignment\Project_01\news-sentence-with-punctuation-cut.txt'

In [6]:
sentence_word_content_path = r'D:\Assignment\Project_01\news-sentence-word-cut.txt'

In [7]:
def generate_word_list(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    article_contents_list = []
    i = 0
    while content_line:
        temp = ''
        content_line = content_line.strip("\n")
        if len(content_line) > 0:
            for word in content_line:
                if word == ' ':
                    article_contents_list += [temp]
                    temp = ''
                else:
                    temp = temp + word
        content_line = file.readline()
        if temp != '': article_contents_list += [temp]
    return article_contents_list

In [8]:
def generate_sentence_list(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    temp_list = []
    article_contents_list = []
    while content_line:
        if content_line == '\n':
            article_contents_list.append(temp_list)
            temp_list = []
        else:
            content_line = content_line.strip('\n')
            temp_list.append(content_line)
        content_line= file.readline()
    return article_contents_list

In [9]:
def generate_sentence_word_list(file_path):
    file = open(file_path, "r", encoding="utf-8")
    content_line = file.readline()
    temp_word = ''
    temp_sentence = []
    temp_news = []
    article_contents_list = []
    while content_line:
        if content_line == '\n':
            article_contents_list.append(temp_news)
            temp_news = []
        else:
            for word in content_line:
                if word == ' ':
                    temp_sentence.append(temp_word)
                    temp_word = ''
                elif word == '\n':
                    temp_news.append(temp_sentence)
                    temp_sentence = []
                else:
                    temp_word = temp_word + word
        content_line= file.readline()
    return article_contents_list

In [10]:
word_content = generate_word_list(word_content_path)

In [11]:
sentence_content = generate_sentence_list(sentence_content_path)

In [12]:
sentence_content_with_punctuation = generate_sentence_list(sentence_content_with_punctuation_path)

In [ ]:
sentence_word_content = generate_sentence_word_list(sentence_word_content_path)

In [13]:
sentence_content_with_punctuation[0]

['此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。',
 '有人猜测这也是将精力主要用到MIUI 9的研发之中。',
 'MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。',
 '当然，关于MIUI 9的确切信息，我们还是等待官方消息。']

In [14]:
sentence_content[0]

['此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作',
 '有人猜测这也是将精力主要用到MIUI9的研发之中',
 'MIUI8去年5月发布距今已有一年有余也是时候更新换代了',
 '当然关于MIUI9的确切信息我们还是等待官方消息']

In [16]:
word_content[0:10]

['此外', '自', '本周', '6', '月', '12', '日起', '除', '小米', '手机']

In [17]:
sentence_word_content[0][0]

['此外',
 '自',
 '本周',
 '6',
 '月',
 '12',
 '日起',
 '除',
 '小米',
 '手机',
 '6',
 '等',
 '15',
 '款',
 '机型',
 '外',
 '其余',
 '机型',
 '已',
 '暂停',
 '更新',
 '发布',
 '含',
 '开发',
 '版',
 '体验版',
 '内测',
 '稳定版',
 '暂不受',
 '影响',
 '以',
 '确保',
 '工程师',
 '可以',
 '集中',
 '全部',
 '精力',
 '进行',
 '系统优化',
 '工作']

In [18]:
import gensim

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
word2vec = gensim.models.word2vec.Word2Vec.load("./Word2vec_model.w2v").wv

In [20]:
word2vec.most_similar('体验版', topn=10)

[('毛用', 0.7442479729652405),
 ('生产工艺', 0.7230020761489868),
 ('肉用', 0.7132026553153992),
 ('因原', 0.7121168375015259),
 ('拙政园', 0.707581639289856),
 ('装配式', 0.7034838199615479),
 ('噪声', 0.6892940402030945),
 ('叶片', 0.6849146485328674),
 ('工艺性能', 0.6845957040786743),
 ('分箱', 0.6816134452819824)]

# 2 Get the set of chinese words associated with ['说', '表示']

In [21]:
from collections import defaultdict

In [22]:
def memo(func):
    cache = {}

    def _wrap(*args): ## ? *args, **kwargs
        if args in cache: result = cache[args]
        else:
            result = func(*args)
            cache[args] = result
        return result
    return _wrap

In [23]:
@memo
def get_related_words_first(input_word, model):
    return [word for word, val in model.most_similar(input_word, topn=20)]

In [24]:
def get_related_words(initial_words, model):
    unseen = initial_words
    
    seen = defaultdict(int)
    
    max_size = 500
    
    while unseen and len(seen) < max_size:
        if len(seen) % 50 == 0: 
            print('seen length : {}'.format(len(seen)))
        
        now_word = unseen.pop(0)
        
        new_append = get_related_words_first(now_word, model)
        
        unseen += new_append
        
        seen[now_word] += 1
    return seen

In [25]:
related_word = get_related_words(['说', '表示'], word2vec)

seen length : 0
seen length : 50
seen length : 100
seen length : 100
seen length : 150
seen length : 200
seen length : 250
seen length : 300
seen length : 350
seen length : 400
seen length : 450


In [26]:
related_word = sorted(related_word.items(), key=lambda x: x[1], reverse=True)

In [51]:
related_word[:10]

[('表示', 93),
 ('说', 86),
 ('认为', 81),
 ('坦言', 75),
 ('透露', 73),
 ('指出', 68),
 ('告诉', 56),
 ('称', 51),
 ('介绍', 48),
 ('称赞', 42)]

In [28]:
related_word_use = [word for word, val in related_word if val> 6]

In [50]:
related_word_use[:10]

['表示', '说', '认为', '坦言', '透露', '指出', '告诉', '称', '介绍', '称赞']

# 3 SIF

## 3.1 Get the probality of word

In [30]:
from collections import Counter

In [31]:
unigrams_dict = Counter(word_content)

word_list_len = len(word_content)

def get_unigrams_probability(word):
    esp = 1 / word_list_len
    if word in unigrams_dict:
        return unigrams_dict[word] / word_list_len 
    else:
        return esp

In [32]:
get_unigrams_probability('啊')

4.5933924963048614e-05

## 3.2 Sentences embdding

In [33]:
import numpy as np

In [34]:
from sklearn.decomposition import PCA

In [35]:
def get_SIF(word, a=0.01):
    return (a + get_unigrams_probability(word))

In [36]:
def SIF_sentence_embdding(sentence, a=0.01):
    result = 0
    i = 0
    for word in sentence:
        temp = word2vec[word] * get_SIF(word)
        result += temp
    return result

In [37]:
def PCA_SIF_sentence_embdding(news, embedding_size=35, a=0.01):
    X = [SIF_sentence_embdding(sentence) for sentence in news]
    
    pca = PCA(n_components=min(embedding_size, len(X)))
    
    pca.fit(np.array(X))
    
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT
    
    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below
    # resulting sentence vectors, vs = vs -u x uT x vs
    Y = []
    for Vs in X:
        sub = np.multiply(u, Vs)
        Y.append(np.subtract(Vs, sub))
    return Y

In [38]:
from collections import defaultdict

In [39]:
def generate_all_PCA_SIF_sentence_embdding(content):
    result = []
    for news in content:
        yield PCA_SIF_sentence_embdding(news)

In [40]:
#sentence_embdding = generate_all_PCA_SIF_sentence_embdding(sentence_word_content)

In [41]:
from scipy.spatial.distance import cosine

In [42]:
def distance(v1, v2):
    return cosine(v1, v2)

# 4 NER & Dependency Prasing

In [43]:
from stanfordcorenlp import StanfordCoreNLP

In [44]:
nlp = StanfordCoreNLP(r'D:\stanford_nlp', lang='zh')

# 5 execute program

In [45]:
from collections import defaultdict

In [47]:
PERSON = []
V_point = defaultdict(list)
for k, news in enumerate(sentence_word_content):
    flag = -1
    
    for j, sentence in enumerate(news):
        for i, word in enumerate(sentence):
            if word in related_word_use:
                flag = i
                break
                
        if flag >= 0:           
            Y = PCA_SIF_sentence_embdding(news)
            
            temp_person = ''
            temp_sen = sentence_content[k][j]
            temp_point = [sentence_content_with_punctuation[k][j]]
            
            NER = nlp.ner(temp_sen)
            D_pr = nlp.dependency_parse(temp_sen)
            
            for i, relation in enumerate(D_pr):
                if relation[1] == flag+1 and relation[0] == 'nsubj' and (NER[relation[-1] - 1][-1] == 'PERSON' or NER[relation[-1] - 1][-1] == 'ORGANIZATION'):
                    temp_person = sentence[relation[-1] - 1]
                    PERSON.append(sentence[relation[-1] - 1])
                    for m in range((len(news) - j -1)):
                        m += (j+1)
                        if(distance(Y[j], Y[m])) < 0.2:
                            temp_point.append(sentence_content_with_punctuation[k][j])
                        else:
                            break
            if temp_person != '':
                V_point[temp_person] = temp_point
        flag = -1

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_variance_ = (S ** 2) / (n_samples - 1)
D:\Anaconda3\lib\site-packages\sklearn\decomposition\pca.py:444: RuntimeWarning: invalid value encountered in true_divide
  explained_v

In [52]:
PERSON[:10]

['刘青', '团队', '证券', '证券', '云龙', '于', '路透社', '孙宏斌', '孙宏斌', '晃']

In [54]:
V_point['刘青']

['刘青介绍，6月16日早上7时36分，他正在环城南路附近值勤，接到中队关于一位女子裸身进入机动车可能有危险的警情，随后骑着小铁骑开始沿路寻找，大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子。']